-- https://community.spotify.com/t5/Spotify-for-Developers/Api-to-create-a-private-playlist-doesn-t-work/m-p/6029675/highlight/true#M13633
> Let's recap once more.
>
> Before, Spotify playlists could be private or public.  Public meant it was published to your user profile, private meant that it was not published to your user profile, however it could still be accessed via the URI/URL.
>
> A few years ago, Spotify improved this for their applications, and introduced the possibility for a playlist to be truly private, which would mean that even if others had the URI/URL, they could not access your playlist.  They decided that this was a better representation of what public/private meant, so in the applications, the public/private toggle would toggle this state instead.
> 
>The previous public/private states, which was more accurately a reflection of whether the playlist was published on your profile, was renamed in the apps to "Publish on profile".
> 
> While Spotify made this change in the applications, they have not (yet) moved this into the Web API functionality, so in the Web API, the public/private boolean represents whether it is published on a users page, and there is no functionality to allow users to toggle the new (true) public/private state.


-- https://developer.spotify.com/documentation/web-api/concepts/playlists
> Note that the public attribute does not refer to access control, modifying access is currently not possible through the WebAPI, so anyone with the link to the playlist can access it unless it’s made private through for instance the desktop client.

In [ ]:
import requests
import streamlit as st
import base64

In [4]:
CLIENT_ID = st.secrets.spotify_app["CLIENT_ID"]
CLIENT_SECRET = st.secrets.spotify_app["CLIENT_SECRET"]

In [21]:
# Request an access token
# https://developer.spotify.com/documentation/web-api/tutorials/getting-started#request-an-access-token
# https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow

url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "client_credentials"
}
# NOTE: relying on `HTTPBAsicAuth` for auto encoding to reduce risk from manual
# construction errors (ex. incorrect encoding, formatting)
response = requests.post(
    url,
    headers=headers,
    data=data,
    auth=requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
)

response.raise_for_status()
access_token = response.json()["access_token"]

In [24]:
playlist_id = ""
r = requests.get(
    f"https://api.spotify.com/v1/me/playlists",
    headers={"Authorization": f"Bearer {access_token}"}
)

r.status_code, r.text

(401,
 '{"error": {"status": 401, "message": "Valid user authentication required" } }')

Scope:
* `playlist-read-private` for /me/playlists
* playlist-modify-public
* playlist-modify-private

In [ ]:
02hD0pKOwQyqhZdd6guYgX

1. https://developer.spotify.com/documentation/web-api/reference/get-current-users-profile
    ```
    ny1ywy0fng5a0mu8ch8u51g20
    ```
2. https://api.spotify.com/v1/me/playlists
    ```
    02hD0pKOwQyqhZdd6guYgX
    ```
3. 


# With Auth Code Flow w/ PKCE

Flow steps
1. PKCE (Proof Key for Code Exchange) flow begins with code verifier
2. Code verifier - high-entropy cryptographic random string between 43 - 128 characters
3. Code challenge is then derived by applying the SHA-256 hashf unction to the code verifier
4. After, encode the result in Base64 URL format

In [26]:
import os
import base64
import hashlib

def generate_code_verifier():
    return base64.urlsafe_b64encode(os.urandom(64)).rstrip(b'=').decode('utf-8')

def generate_code_challenge(code_verifier):
    code_verifier_bytes = code_verifier.encode('utf-8')
    code_challenge_digest = hashlib.sha256(code_verifier_bytes).digest()
    return base64.urlsafe_b64encode(code_challenge_digest).rstrip(b'=').decode('utf-8')

code_verifier = generate_code_verifier()
code_challenge = generate_code_challenge(code_verifier)


5. Redirect the user to Spotify's authorization endpoint

In [ ]:
import urllib.parse
import streamlit as st

CLIENT_ID = st.secrets.spotify_app["CLIENT_ID"]
CLIENT_SECRET = st.secrets.spotify_app["CLIENT_SECRET"]
redirect_uri = 'http://localhost:8888/callback'
# REVIEW: probably too broad scope for prod
scope = "user-read-private user-read-email user-library-read playlist-read-private playlist-read-collaborative playlist-modify-private playlist-modify-public"
state = 'some_random_state'  # Optional but recommended

auth_params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': scope,
    'redirect_uri': redirect_uri,
    'state': state,
    'code_challenge_method': 'S256',
    'code_challenge': code_challenge
}

auth_url = 'https://accounts.spotify.com/authorize?' + urllib.parse.urlencode(auth_params)
print(f'Please go to the following URL to authorize, then copy the URL you\'re sent to after confirmation:\n{auth_url}')

Please go to the following URL to authorize:
https://accounts.spotify.com/authorize?response_type=code&client_id=0226e82b1ca04230becd1b23e697931a&scope=user-read-private+user-read-email+user-library-read+playlist-read-private+playlist-read-collaborative+playlist-modify-private+playlist-modify-public&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&state=some_random_state&code_challenge_method=S256&code_challenge=2i-8KKtORMYvaRZKQ4ANsEZaA9zwlG9tQ1zgvtVwXhc


In [28]:
redirect_resp = input("Enter the redirect URL: ")

In [30]:
# Extract the authorization code from the URL
parsed_url = urllib.parse.urlparse(redirect_resp)
auth_code = urllib.parse.parse_qs(parsed_url.query)['code'][0]

In [31]:
auth_code

'AQDDFzkHerGIIlRHG0McLG7G8dfCx349iK7cZFBmQqfTuRp4PHYgtsbLQKIMmcMhkR8p8jPdHk91rTj6RMuJ6mHqZpZRy3xtU1Ioe24NVncj-Bp8Mc1H6VpHgJnlYPXW_t2suXOZpCGuZXtpMqf0Vf521QjFMtLv9uoRRWgM4N5T-1qcHWnpb0E3lI4HL34Dm55_iUPPOGBItv4JAeUz13RUeNOxeRgqkeIfISAb1ozJ0O7q5jd0dd-HXOuUaF5K7f-xTz2JWfNvt1ABhTMoSs55cV39VvQxBw6aLTKHWDcR_Uuc1JZED93NayGR0TKXRY67VkDbkiOepI9ohaUbhBfaPI0gD4SwiUDgNSktCI1Ivxg5zdqb-sphXY0LhbFakqbleuxg7-85YKqUNEW-cJ1XrrgcSPyk9cE3iG0aXEstf_yVyoLNZzrOGeuDXOHZUQ'

In [33]:
# Step 3: Exchange Authorization Code for Access Token
token_url = 'https://accounts.spotify.com/api/token'
token_data = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': redirect_uri,
    'client_id': CLIENT_ID,
    'code_verifier': code_verifier
}

response = requests.post(token_url, data=token_data)
tokens = response.json()
access_token = tokens.get('access_token')

print(f'Access Token: {access_token}')


Access Token: BQA6fVnUR3YJhoFsI7GFmcEGQvMfR6YqZ4_VoiHwErUwDYeKvV_cWBxcjB4Ev9yW8Q5ZhLV_fioc9ytCD3MY2kPQV4_d-Kqv9r0DIEqpIROcUnL1HKRrQkaKYtu6rbUCICtfxGCX4e5oh-x2z_poAS8_u7zav3MccvDKh-opyiDuTvdaXctlvdvAD0CeSBamaYw7_g2JChOJNzJXVObuy_d5HQWD8boF8ONIxahk2Qkd1V0Q8ntXjSN_Q5RRPVmrfgFbr1ikmBCJBurUCwtT05q4aXDLGGYmjmPMNXVmRHce8gqP


In [35]:
r = requests.get(
    'https://api.spotify.com/v1/me/playlists',
    headers={
        'Authorization': f'Bearer {access_token}'
    }
)
r.status_code, r.text

(200,
 '{"href":"https://api.spotify.com/v1/users/ny1ywy0fng5a0mu8ch8u51g20/playlists?offset=0&limit=50","limit":50,"next":"https://api.spotify.com/v1/users/ny1ywy0fng5a0mu8ch8u51g20/playlists?offset=50&limit=50","offset":0,"previous":null,"total":1002,"items":[null,null,{"collaborative":false,"description":"test","external_urls":{"spotify":"https://open.spotify.com/playlist/02hD0pKOwQyqhZdd6guYgX"},"href":"https://api.spotify.com/v1/playlists/02hD0pKOwQyqhZdd6guYgX","id":"02hD0pKOwQyqhZdd6guYgX","images":[{"height":640,"url":"https://mosaic.scdn.co/640/ab67616d00001e022e403793b2ea0101ec26cb11ab67616d00001e024a7d1a57a1e7b971d53e5bd5ab67616d00001e02967b5ff9a544514b07df3db2ab67616d00001e02f9a38d43d4cf639296987b53","width":640},{"height":300,"url":"https://mosaic.scdn.co/300/ab67616d00001e022e403793b2ea0101ec26cb11ab67616d00001e024a7d1a57a1e7b971d53e5bd5ab67616d00001e02967b5ff9a544514b07df3db2ab67616d00001e02f9a38d43d4cf639296987b53","width":300},{"height":60,"url":"https://mosaic.scdn.co

In [43]:
resp_data = r.json()
# Loop through response data to find the first playlist with `public: True`
for playlist in resp_data["items"]:
    if playlist is not None:
        if playlist["public"]:
            playlist_id = playlist["id"]
            playlist_name = playlist["name"]
            break

playlist_id, playlist_name

('02hD0pKOwQyqhZdd6guYgX', 'test')

In [44]:
r = requests.put(
    f'https://api.spotify.com/v1/playlists/{playlist_id}',
    headers={
        'Authorization': f'Bearer {access_token}'
    },
    data={
        'public': False
    }
)
r.status_code, r.text

(200, '')

In [46]:
r = requests.get(
    f'https://api.spotify.com/v1/playlists/{playlist_id}',
    headers={
        'Authorization': f'Bearer {access_token}'
    }
)
r.status_code, r.text, r.json()['public']

(200,
 '{"collaborative":false,"description":"test","external_urls":{"spotify":"https://open.spotify.com/playlist/02hD0pKOwQyqhZdd6guYgX"},"followers":{"href":null,"total":0},"href":"https://api.spotify.com/v1/playlists/02hD0pKOwQyqhZdd6guYgX","id":"02hD0pKOwQyqhZdd6guYgX","images":[{"height":640,"url":"https://mosaic.scdn.co/640/ab67616d00001e022e403793b2ea0101ec26cb11ab67616d00001e024a7d1a57a1e7b971d53e5bd5ab67616d00001e02967b5ff9a544514b07df3db2ab67616d00001e02f9a38d43d4cf639296987b53","width":640},{"height":300,"url":"https://mosaic.scdn.co/300/ab67616d00001e022e403793b2ea0101ec26cb11ab67616d00001e024a7d1a57a1e7b971d53e5bd5ab67616d00001e02967b5ff9a544514b07df3db2ab67616d00001e02f9a38d43d4cf639296987b53","width":300},{"height":60,"url":"https://mosaic.scdn.co/60/ab67616d00001e022e403793b2ea0101ec26cb11ab67616d00001e024a7d1a57a1e7b971d53e5bd5ab67616d00001e02967b5ff9a544514b07df3db2ab67616d00001e02f9a38d43d4cf639296987b53","width":60}],"name":"test","owner":{"display_name":"Jordie","e